In [1]:
import pandas as pd
from scipy.io import arff
import numpy as np
from sklearn.model_selection import train_test_split
import erlc as e
import dataPipeline as dp
from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef
from sklearn.model_selection import learning_curve
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import random


In [2]:
dataPipeline = dp.DataPipeline()
data = dataPipeline.loadData()
X, y = dataPipeline.dataProc(data)

Loading dataset #1
Loading dataset #2
Loading dataset #3
Loading dataset #4
Loading dataset #5
Loading dataset #6
Loading dataset #7
Loading dataset #8
Loading dataset #9
Loading dataset #10
Loading dataset #11
Loading dataset #12
Loading dataset #13
Loading dataset #14
Loading dataset #15
Finished Loading. Final Size = (78377, 129)


In [3]:
X.shape

(78377, 117)

In [4]:
# train_sizes, train_scores, test_scores, fit_times, score_times = learning_curve(model,
#                                                                       X, y, 
#                                                                       cv=2,
#                                                                       scoring = "accuracy",
#                                                                       return_times=True)

In [5]:
def evaluateModel (model, X, y, feature_percentage = 1.0):
    
    # Extract random features from data
    num_features = round(feature_percentage * X.shape[1]).astype(int)
    print("NUM FEATURES = {}".format(num_features))

    idx = random.sample(range(X.shape[1]), num_features)
    idx = np.sort(idx)
    X = X[:, idx]
    print("Testing on {} features".format(X.shape[1]))

    # Splitting data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # Training model
    time1 = time.time()
    model.fit(X_train, y_train)
    time2 = time.time()
    train_time = time2 - time1
    
    # Testing model
    time1 = time.time()
    y_pred = model.predict(X_test)
    time2 = time.time()
    test_time = time2 - time1
    
    # Calculating metrics
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    mcc = matthews_corrcoef(y_test, y_pred)
    print("Accuracy = {}, F1-score = {}, MCC = {}".format(acc, f1, mcc))
    
    return train_time, test_time, acc, f1, mcc

In [6]:
def scalabilityAnalysis (model, X, y, feature_percentages = np.linspace(0.1,1, 10) ):
    train_times = []
    test_times = []
    acc_scores = []
    f1_scores = []
    mcc_scores = []
    
    for fp in feature_percentages:
        train_time, test_time, acc, f1, mcc = evaluateModel(model, X, y, feature_percentage = fp)
        train_times.append(train_time)
        test_times.append(test_time)
        acc_scores.append(acc)
        f1_scores.append(f1)
        mcc_scores.append(mcc)
        
    return train_times, test_times, acc_scores, f1_scores, mcc_scores

In [7]:
modelDT = DecisionTreeClassifier()
modelRF = RandomForestClassifier()
modelKNN = KNeighborsClassifier()
modelERLC = e.ERLC()
MODELS = [modelDT, modelRF, modelKNN, modelERLC]

In [10]:
# Feature percentages for scalability test
# fp_range = np.arange(0.2,1.2,0.2)
fp_range = np.arange(0.1,1.1,0.1)

writer = pd.ExcelWriter('output/Results_scalabilityAnalysis_{}intervals.xlsx'.format(len(fp_range)))

for model in MODELS:
    model_name = model.__class__.__name__
    print(model_name)
    
    # Perform scalability analysis on current model
    train_times, test_times, acc_scores, f1_scores, mcc_scores = \
     scalabilityAnalysis(model, X, y, feature_percentages = fp_range )
    
    
    # Add to excel file
    df = pd.DataFrame({
        'Feature Percentage': fp_range,
        'Accuracy': acc_scores,
        'F1-Score': f1_scores,
        'MCC': mcc_scores,
        'Training Time': train_times,
        'Testing Time': test_times
    })
    
    df.to_excel(writer, model_name)
    
    
    
writer.save()

DecisionTreeClassifier
NUM FEATURES = 12
Testing on 12 features
Accuracy = 0.6775963255932635, F1-score = 0.6781705678224477, MCC = 0.6669775863888799
NUM FEATURES = 23
Testing on 23 features
Accuracy = 0.9106277111508038, F1-score = 0.910556154032084, MCC = 0.9077095668909447
NUM FEATURES = 35
Testing on 35 features
Accuracy = 0.8972952283745853, F1-score = 0.897235808486582, MCC = 0.893921448620119
NUM FEATURES = 47
Testing on 47 features
Accuracy = 0.7116611380454197, F1-score = 0.711840341590065, MCC = 0.7020725985366683
NUM FEATURES = 58
Testing on 58 features
Accuracy = 0.7388364378668028, F1-score = 0.738850356816104, MCC = 0.7302717978919596
NUM FEATURES = 70
Testing on 70 features
Accuracy = 0.7352003062005613, F1-score = 0.7351347705346377, MCC = 0.7265714864656905
NUM FEATURES = 82
Testing on 82 features
Accuracy = 0.901952028578719, F1-score = 0.9019929319487185, MCC = 0.8987346465419364
NUM FEATURES = 94
Testing on 94 features
Accuracy = 0.902398571064047, F1-score = 0.902

196/196 [==============================] - 4s 20ms/step - loss: 3.2480 - acc: 0.1106 - f1_m: 0.0166 - val_loss: 3.2446 - val_acc: 0.1103 - val_f1_m: 0.0146
Epoch 10/500
196/196 [==============================] - 4s 19ms/step - loss: 3.2169 - acc: 0.1190 - f1_m: 0.0179 - val_loss: 3.2142 - val_acc: 0.1160 - val_f1_m: 0.0203
Epoch 11/500
196/196 [==============================] - 3s 17ms/step - loss: 3.2002 - acc: 0.1194 - f1_m: 0.0204 - val_loss: 3.2045 - val_acc: 0.1167 - val_f1_m: 0.0152
Epoch 12/500
196/196 [==============================] - 3s 16ms/step - loss: 3.1816 - acc: 0.1230 - f1_m: 0.0211 - val_loss: 3.1922 - val_acc: 0.1214 - val_f1_m: 0.0186
Epoch 13/500
196/196 [==============================] - 3s 15ms/step - loss: 3.1699 - acc: 0.1235 - f1_m: 0.0227 - val_loss: 3.1657 - val_acc: 0.1252 - val_f1_m: 0.0209
Epoch 14/500
196/196 [==============================] - 3s 14ms/step - loss: 3.1405 - acc: 0.1284 - f1_m: 0.0235 - val_loss: 3.1549 - val_acc: 0.1268 - val_f1_m: 0.0211

196/196 [==============================] - 4s 19ms/step - loss: 2.5983 - acc: 0.2274 - f1_m: 0.0632 - val_loss: 2.7097 - val_acc: 0.2139 - val_f1_m: 0.0479
Epoch 58/500
196/196 [==============================] - 4s 19ms/step - loss: 2.5835 - acc: 0.2312 - f1_m: 0.0661 - val_loss: 2.6961 - val_acc: 0.2202 - val_f1_m: 0.0499
Epoch 59/500
196/196 [==============================] - 3s 17ms/step - loss: 2.5904 - acc: 0.2297 - f1_m: 0.0647 - val_loss: 2.7012 - val_acc: 0.2166 - val_f1_m: 0.0470
Epoch 60/500
196/196 [==============================] - 3s 16ms/step - loss: 2.5771 - acc: 0.2311 - f1_m: 0.0654 - val_loss: 2.6894 - val_acc: 0.2195 - val_f1_m: 0.0552
Epoch 61/500
196/196 [==============================] - 3s 14ms/step - loss: 2.5660 - acc: 0.2326 - f1_m: 0.0697 - val_loss: 2.6887 - val_acc: 0.2185 - val_f1_m: 0.0578
Epoch 62/500
196/196 [==============================] - 2s 12ms/step - loss: 2.5553 - acc: 0.2360 - f1_m: 0.0688 - val_loss: 2.6834 - val_acc: 0.2268 - val_f1_m: 0.0490

196/196 [==============================] - 2s 13ms/step - loss: 2.2334 - acc: 0.3075 - f1_m: 0.1324 - val_loss: 2.5393 - val_acc: 0.2635 - val_f1_m: 0.0958
Epoch 106/500
196/196 [==============================] - 5s 24ms/step - loss: 2.2039 - acc: 0.3161 - f1_m: 0.1354 - val_loss: 2.4375 - val_acc: 0.2870 - val_f1_m: 0.1178
Epoch 107/500
196/196 [==============================] - 4s 19ms/step - loss: 2.2016 - acc: 0.3173 - f1_m: 0.1405 - val_loss: 2.4636 - val_acc: 0.2767 - val_f1_m: 0.0946
Epoch 108/500
196/196 [==============================] - 4s 19ms/step - loss: 2.1933 - acc: 0.3203 - f1_m: 0.1410 - val_loss: 2.4129 - val_acc: 0.2891 - val_f1_m: 0.1184
Epoch 109/500
196/196 [==============================] - 3s 17ms/step - loss: 2.1956 - acc: 0.3165 - f1_m: 0.1411 - val_loss: 2.4539 - val_acc: 0.2745 - val_f1_m: 0.1010
Epoch 110/500
196/196 [==============================] - 3s 16ms/step - loss: 2.1814 - acc: 0.3241 - f1_m: 0.1460 - val_loss: 2.4091 - val_acc: 0.2939 - val_f1_m: 0

196/196 [==============================] - 3s 16ms/step - loss: 1.8903 - acc: 0.4024 - f1_m: 0.2340 - val_loss: 2.2289 - val_acc: 0.3640 - val_f1_m: 0.1793
Epoch 154/500
196/196 [==============================] - 3s 15ms/step - loss: 1.9105 - acc: 0.3939 - f1_m: 0.2345 - val_loss: 2.6332 - val_acc: 0.2405 - val_f1_m: 0.1445
Epoch 155/500
196/196 [==============================] - 3s 14ms/step - loss: 1.8726 - acc: 0.4104 - f1_m: 0.2409 - val_loss: 2.2040 - val_acc: 0.3687 - val_f1_m: 0.1950
Epoch 156/500
196/196 [==============================] - 3s 14ms/step - loss: 1.8656 - acc: 0.4080 - f1_m: 0.2440 - val_loss: 2.3090 - val_acc: 0.3230 - val_f1_m: 0.1820
Epoch 157/500
196/196 [==============================] - 5s 26ms/step - loss: 1.8776 - acc: 0.4044 - f1_m: 0.2410 - val_loss: 2.2071 - val_acc: 0.3646 - val_f1_m: 0.1882
Epoch 158/500
196/196 [==============================] - 4s 19ms/step - loss: 1.8696 - acc: 0.4053 - f1_m: 0.2460 - val_loss: 2.3839 - val_acc: 0.3234 - val_f1_m: 0

...

196/196 [==============================] - 1s 4ms/step - loss: 4.3721e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.7511e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 280/500
196/196 [==============================] - 1s 4ms/step - loss: 4.3001e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.8461e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 281/500
196/196 [==============================] - 1s 4ms/step - loss: 4.3146e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.8024e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 282/500
196/196 [==============================] - 1s 3ms/step - loss: 4.3004e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.7438e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 283/500
196/196 [==============================] - 1s 4ms/step - loss: 4.2996e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.9235e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 284/500
196/196 [==============================] - 1s 4ms/step - loss: 4.3408e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 

196/196 [==============================] - 1s 4ms/step - loss: 4.1006e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.4162e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 326/500
196/196 [==============================] - 1s 4ms/step - loss: 3.9377e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.3839e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 327/500
196/196 [==============================] - 1s 3ms/step - loss: 3.9050e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.4579e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 328/500
196/196 [==============================] - 1s 4ms/step - loss: 3.9233e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.3656e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 329/500
196/196 [==============================] - 1s 3ms/step - loss: 3.9104e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.4020e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 330/500
196/196 [==============================] - 1s 3ms/step - loss: 3.8931e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 

196/196 [==============================] - 1s 3ms/step - loss: 3.6758e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.2375e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 372/500
196/196 [==============================] - 1s 3ms/step - loss: 3.6617e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.1814e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 373/500
196/196 [==============================] - 1s 3ms/step - loss: 3.6753e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.0629e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 374/500
196/196 [==============================] - 2s 13ms/step - loss: 3.6433e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.1344e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 375/500
196/196 [==============================] - 3s 15ms/step - loss: 3.6308e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.0945e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 376/500
196/196 [==============================] - 2s 10ms/step - loss: 3.6521e-04 - acc: 1.0000 - f1_m: 1.0000 - val_los

196/196 [==============================] - 1s 3ms/step - loss: 3.4627e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 3.8090e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 418/500
196/196 [==============================] - 1s 3ms/step - loss: 3.4178e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 3.8369e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 419/500
196/196 [==============================] - 1s 3ms/step - loss: 3.4230e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 3.8372e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 420/500
196/196 [==============================] - 1s 3ms/step - loss: 3.4258e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 3.8414e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 421/500
196/196 [==============================] - 1s 3ms/step - loss: 3.4509e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 3.8018e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 422/500
196/196 [==============================] - 1s 3ms/step - loss: 3.4088e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 

196/196 [==============================] - 1s 4ms/step - loss: 3.2521e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 3.6222e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 464/500
196/196 [==============================] - 1s 4ms/step - loss: 3.2400e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 3.6477e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 465/500
196/196 [==============================] - 1s 4ms/step - loss: 3.3879e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 3.5916e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 466/500
196/196 [==============================] - 1s 4ms/step - loss: 3.2325e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 3.6426e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 467/500
196/196 [==============================] - 1s 4ms/step - loss: 3.2290e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 3.6637e-04 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 468/500
196/196 [==============================] - 1s 4ms/step - loss: 3.2401e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 